# Part 1: Data Scraping & Cleaning

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Problem-Statement" data-toc-modified-id="Problem-Statement-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Problem Statement</a></span></li><li><span><a href="#Data-Acquisition" data-toc-modified-id="Data-Acquisition-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Acquisition</a></span></li><li><span><a href="#Importing-Libraries" data-toc-modified-id="Importing-Libraries-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Importing Libraries</a></span></li><li><span><a href="#Data-Scraping-w/-Beautiful-Soup" data-toc-modified-id="Data-Scraping-w/-Beautiful-Soup-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Scraping w/ Beautiful Soup</a></span></li><li><span><a href="#Scraped-Data" data-toc-modified-id="Scraped-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Scraped Data</a></span></li><li><span><a href="#Data-Cleaning" data-toc-modified-id="Data-Cleaning-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Data Cleaning</a></span></li></ul></div>

## Problem Statement

Film production is a huge financial risk. Currently, blockbuster movies can have budgets in excess of of hundreds of millions of dollars. This risk could be reduced to an extent with the use of quantitative modeling. Since movie scripts are pitched and budgets are set and spent well before any revenue is ever made, we plan to construct a predictive model using features that are generally known before the movie release date such as the movie length, budget, and genre.

## Data Acquisition

We used IMDB (https://www.imdb.com/search/title/?title_type=feature&release_date=1972-01-01,2020-01-10&count=250) to acquire various information related to a given movie released between 1972 to 2020. We used BeautifulSoup to scrape data (movie titles, released year, ratings, metascore, votes by IMDB users, length, genre, and so on) from each page. 

## Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import requests
from requests import get
from bs4 import BeautifulSoup
from IPython.core.display import display,HTML
from time import sleep
from random import randint

sns.set()
%matplotlib inline

In [2]:
pd.set_option('precision', 2)

## Data Scraping w/ Beautiful Soup

In [562]:
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
lengths = []
genres = []
ratings = []
movie_ids = []

pages = [str(i) for i in range(1,177)]
headers = {"Accept-Language": "en-US, en;q=0.5"}

for page in pages:
    response = get('https://www.imdb.com/search/title/?title_type=feature&release_date=1972-01-01,2019-12-31&count=250&start='+page+'&ref_=adv_nxt', headers = headers)        
    
    # Adding a pause to the loop
    sleep(randint(8,15))
    requests += 1

    html_soup = BeautifulSoup(response.text, 'html.parser')
    movie_list = html_soup.find_all('div', class_ = 'lister-item mode-advanced')

    # Extract data from individual movie list
    for movie in movie_list:
    # If the movie has Metascore, then extract:
        if movie.find('div', class_ = 'ratings-metascore') is not None:
            
    # The name
            name = movie.h3.a.text
            names.append(name)
            
    # The year
            year = movie.h3.find('span', class_ = 'lister-item-year').text
            years.append(year)

    # The Genres
            genre = movie.find('span', class_ = 'genre').text
            genres.append(genre[1:])

    # The IMDB rating
            imdb = float(movie.strong.text)
            imdb_ratings.append(imdb)
            
    # The Movie Length
            length = movie.find('span', class_ = 'runtime').text
            lengths.append((length))
            
    # The Metascores
            m_score = movie.find('span', class_ = 'metascore').text
            metascores.append(int(m_score))
    
    # The Ratings
            rating = movie.find('span', class_ = 'certificate').text
            ratings.append(rating)

    # Movie IDs
            movie_id = movie.h3.find('a')['href']
            movie_ids.append(movie_id[9:-1])
            
    # The number of votes
            vote = movie.find('span', attrs = {'name':'nv'})['data-value']
            votes.append(int(vote))

# Convert Lists to Dataframe            
movie_dataframe = pd.DataFrame({'movie_id':movie_ids,'movie':names,'year':years,'imdb':imdb_ratings,'metascore':metascores,'votes':votes,'lengths':lengths,'ratings':ratings,'genres':genres})


## Scraped Data

In [574]:
movie_dataframe

,movie_id,movie,year,imdb,metascore,votes,lengths,ratings,genres
0,8579674,1917,(2019),8.6,79,64668,119 min,R,"Drama, War"
1,2527338,Star Wars: Episode IX - The Rise of Skywalker,(2019),6.9,54,233251,142 min,PG-13,"Action, Adventure, Fantasy"
2,7131622,Once Upon a Time... in Hollywood,(2019),7.8,83,341435,161 min,R,"Comedy, Drama"
3,7286456,Joker,(2019),8.6,59,618468,122 min,R,"Crime, Drama, Thriller"
4,8946378,Knives Out,(2019),8.1,82,122999,131 min,PG-13,"Comedy, Crime, Drama"
...,...,...,...,...,...,...,...,...,...
42939,2562232,Birdman or (The Unexpected Virtue of Ignorance),(2014),7.7,87,541646,119 min,R,"Comedy, Drama"
42940,0114709,Toy Story,(1995),8.3,95,832172,81 min,G,"Animation, Adventure, Comedy"
42941,0114814,The Usual Suspects,(1995),8.5,77,936438,106 min,R,"Crime, Mystery, Thriller"
42942,2347569,Frances Ha,(2012),7.4,82,64696,86 min,R,"Comedy, Drama, Romance"


__Saving Dataframe as CSV File:__

In [576]:
movie_dataframe.to_csv('../movie_dataframe.csv')

__Reading Saved CSV File:__

In [4]:
df = pd.read_csv('../movie_dataframe.csv')
df.head()

,Unnamed: 0,movie_id,movie,year,imdb,metascore,votes,lengths,ratings,genres
0,0,8579674,1917,(2019),8.6,79,64668,119 min,R,"Drama, War"
1,1,2527338,Star Wars: Episode IX - The Rise of Skywalker,(2019),6.9,54,233251,142 min,PG-13,"Action, Adventure, Fantasy"
2,2,7131622,Once Upon a Time... in Hollywood,(2019),7.8,83,341435,161 min,R,"Comedy, Drama"
3,3,7286456,Joker,(2019),8.6,59,618468,122 min,R,"Crime, Drama, Thriller"
4,4,8946378,Knives Out,(2019),8.1,82,122999,131 min,PG-13,"Comedy, Crime, Drama"


__Scraping Budget, Country, Language, Gross From The IMDB Website, One Page At A Time:__

In [ ]:
budget_list = []
country_list = []
language_list = []
gross_list = []
counter = 0

for title in df['movie_id'].iloc[15000:20000]: 
    counter = counter+1
    print ("Counter:", counter)
    print ("% Completed:", (counter/(len(df['movie_id'].iloc[15000:20000])))*100)

    response = get('https://www.imdb.com/title/tt'+str(title)+'/?ref_=adv_li_tt')
    # Adding a pause to the loop
    sleep(randint(0,2))
    html_soup = BeautifulSoup(response.text, 'html.parser')
    text_box=html_soup.find_all('div',class_="txt-block")
    
    if (html_soup.find(text="Budget:") is None):
        budget_list.append('NA')
    
    if (html_soup.find(text="Country:") is None):
        country_list.append('NA')
        
    if (html_soup.find(text="Language:") is None):
        language_list.append('NA')
    
    if (html_soup.find(text="Cumulative Worldwide Gross:") is None):
        gross_list.append('NA')
    for c in text_box:
        try:
            heading=c.find('h4').getText()
            if heading=="Country:":
                c1=c.find_all('a')
                list1=[]
                if len(c1)>0:
                    for i in range(len(c1)):
                        print (c1[i].text)
                        list1.append(c1[i].text)
                    print (list1)
                    country_list.append(list1)
                if len(c1)==0:
                    list1.append('NA')
                    print (list1)
                    country_list.append(list1)
            if heading=="Language:":
                c2=c.find_all('a')
                list2=[]
                if len(c2)>0:
                    for i in range(len(c2)):
                        print (c2[i].text)
                        list2.append(c2[i].text)
                    print (list2)
                    language_list.append(list2)
                if len(c2)==0:
                    list2.append('NA')
                    print (list2)
                    language_list.append(list2)  
            if heading=="Budget:":
                c3=c.find('h4').next_sibling
                if len(c3)>0:
                    c4=c3.rstrip()
                    print (c4)
                    budget_list.append(c4)
                if len(c3)==0:
                    budget_list.append('NA')  
            if heading=="Cumulative Worldwide Gross:":
                c5=c.find('h4').next_sibling
                if len(c5)>0:
                    c6=c5.rstrip()
                    print (c6)
                    gross_list.append(c5)
                else:
                    gross_list.append('NA')             
        except:
            continue

Counter: 1
UK
USA
['UK', 'USA']
English
French
German
['English', 'French', 'German']
$100,000,000
 $143,336,984
Counter: 2
USA
['USA']
English
['English']
$200,000,000
 $1,028,754,801
Counter: 3
USA
UK
China
['USA', 'UK', 'China']
English
Italian
Spanish
['English', 'Italian', 'Spanish']
$90,000,000
 $373,058,543
Counter: 4
Canada
USA
['Canada', 'USA']
English
['English']
$55,000,000
 $1,069,147,201
Counter: 5
USA
['USA']
English
Spanish
['English', 'Spanish']
$40,000,000
 $277,992,387
Counter: 6
USA
['USA']
English
French
['English', 'French']
$40,000,000
 $130,991,052
Counter: 7
South Korea
['South Korea']
Korean
English
['Korean', 'English']
 $139,600,731
Counter: 8
UK
USA
['UK', 'USA']
English
['English']
$95,000,000
 $60,027,835
Counter: 9
USA
['USA']
English
['English']
 $44,928,599
Counter: 10
USA
['USA']
English
Italian
Latin
Spanish
['English', 'Italian', 'Latin', 'Spanish']
$159,000,000
 $961,224
Counter: 11
UK
USA
['UK', 'USA']
English
['English']
$18,600,000
 $323,382
Coun

Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x1c21e55b90>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/weakref.py", line 109, in remove
    def remove(wr, selfref=ref(self), _atomic_removal=_remove_dead_weakref):
KeyboardInterrupt


## Data Cleaning

Furthermore, we clean the dataframe such that we can perform analysis/modeling with it. Some of the cleaning steps consist of the following:
1. Reordering the columns
2. Cleaning the year column and converting the values to integers

In [4]:
movie_df= pd.read_csv('../movie_dataframe2000movies.csv')

In [5]:
movie_df=movie_df.drop(columns='budget_list')

In [7]:
movie_df.head()

,Unnamed: 0,Unnamed: 0.1,movie_id,movie,year,imdb,metascore,votes,lengths,ratings,genres,budget,language,country,cumulative_worldwide_gross
0,0,0,8579674,1917,(2019),8.6,79,64668,119 min,R,"Drama, War","$100,000,000","['English', 'French', 'German']","['UK', 'USA']","$143,336,984"
1,1,1,2527338,Star Wars: Episode IX - The Rise of Skywalker,(2019),6.9,54,233251,142 min,PG-13,"Action, Adventure, Fantasy","$200,000,000",['English'],['USA'],"$1,028,754,801"
2,2,2,7131622,Once Upon a Time... in Hollywood,(2019),7.8,83,341435,161 min,R,"Comedy, Drama","$90,000,000","['English', 'Italian', 'Spanish']","['USA', 'UK', 'China']","$373,058,543"
3,3,3,7286456,Joker,(2019),8.6,59,618468,122 min,R,"Crime, Drama, Thriller","$55,000,000",['English'],"['Canada', 'USA']","$1,069,147,201"
4,4,4,8946378,Knives Out,(2019),8.1,82,122999,131 min,PG-13,"Comedy, Crime, Drama","$40,000,000","['English', 'Spanish']",['USA'],"$277,992,387"


In [8]:
movie_df1= pd.read_csv('../movie_dataframe2000-5000movies.csv')

In [20]:
movie_df2= pd.read_csv('../movie_dataframe5000-10000movies.csv')

In [10]:
movie_df3= pd.read_csv('../movie_dataframe10000-11000movies.csv')

In [11]:
movie_df4= pd.read_csv('../movie_dataframe11000-15000movies.csv')

In [12]:
movie_df5= pd.read_csv('../movie_dataframe15000-18646movies.csv')

In [13]:
movie_df6= pd.read_csv('../movie_dataframe18647-21000movies.csv')

In [14]:
movie_df7 = pd.read_csv('../movie_dataframe21000-24000movies.csv')

In [15]:
movie_df8 = pd.read_csv('../movie_dataframe24000-27000movies.csv')

In [16]:
movie_df9 = pd.read_csv('../movie_dataframe27000-37000movies.csv')

In [17]:
movie_df10 = pd.read_csv('../movie_dataframe37000-40000movies.csv')

__Concatenating All (40000) Of The Scraped Movies:__

In [28]:
dataframes = [movie_df,movie_df1,movie_df2,movie_df3,movie_df4,movie_df5,movie_df6,movie_df7,movie_df8,movie_df9,movie_df10]

In [29]:
movie_dataframes = pd.concat(dataframes)

In [31]:
movie_dataframes.to_csv('../movie_dataframe_completemovies.csv')

In [18]:
movie_dataframes= pd.read_csv('../movie_dataframe_completemovies.csv')

In [19]:
movie_dataframes.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,movie_id,movie,year,imdb,metascore,votes,lengths,ratings,genres,budget,language,country,cumulative_worldwide_gross
0,0,0,0,8579674,1917,(2019),8.6,79,64668,119 min,R,"Drama, War","$100,000,000","['English', 'French', 'German']","['UK', 'USA']","$143,336,984"
1,1,1,1,2527338,Star Wars: Episode IX - The Rise of Skywalker,(2019),6.9,54,233251,142 min,PG-13,"Action, Adventure, Fantasy","$200,000,000",['English'],['USA'],"$1,028,754,801"
2,2,2,2,7131622,Once Upon a Time... in Hollywood,(2019),7.8,83,341435,161 min,R,"Comedy, Drama","$90,000,000","['English', 'Italian', 'Spanish']","['USA', 'UK', 'China']","$373,058,543"
3,3,3,3,7286456,Joker,(2019),8.6,59,618468,122 min,R,"Crime, Drama, Thriller","$55,000,000",['English'],"['Canada', 'USA']","$1,069,147,201"
4,4,4,4,8946378,Knives Out,(2019),8.1,82,122999,131 min,PG-13,"Comedy, Crime, Drama","$40,000,000","['English', 'Spanish']",['USA'],"$277,992,387"


In [20]:
movie_dataframes=movie_dataframes.drop(columns='Unnamed: 0')
movie_dataframes=movie_dataframes.drop(columns='Unnamed: 0.1')
movie_dataframes=movie_dataframes.drop(columns='Unnamed: 0.1.1')

In [22]:
movie_dataframes.head()

,movie_id,movie,year,imdb,metascore,votes,lengths,ratings,genres,budget,language,country,cumulative_worldwide_gross
0,8579674,1917,(2019),8.6,79,64668,119 min,R,"Drama, War","$100,000,000","['English', 'French', 'German']","['UK', 'USA']","$143,336,984"
1,2527338,Star Wars: Episode IX - The Rise of Skywalker,(2019),6.9,54,233251,142 min,PG-13,"Action, Adventure, Fantasy","$200,000,000",['English'],['USA'],"$1,028,754,801"
2,7131622,Once Upon a Time... in Hollywood,(2019),7.8,83,341435,161 min,R,"Comedy, Drama","$90,000,000","['English', 'Italian', 'Spanish']","['USA', 'UK', 'China']","$373,058,543"
3,7286456,Joker,(2019),8.6,59,618468,122 min,R,"Crime, Drama, Thriller","$55,000,000",['English'],"['Canada', 'USA']","$1,069,147,201"
4,8946378,Knives Out,(2019),8.1,82,122999,131 min,PG-13,"Comedy, Crime, Drama","$40,000,000","['English', 'Spanish']",['USA'],"$277,992,387"


In [41]:
movie_dataframes['year']=movie_dataframes.year.str[-5:-1].astype('int64')

In [45]:
movie_dataframes['lengths']=movie_dataframes.lengths.str[:-4].astype('int64')

In [90]:
movie_dataframes.dtypes

Unnamed: 0                       int64
Unnamed: 0.1                     int64
movie_id                         int64
movie                           object
year                             int64
imdb                           float64
metascore                        int64
votes                            int64
lengths                          int64
ratings                         object
genres                          object
budget                          object
language                        object
country                         object
cumulative_worldwide_gross      object
new_imdb                       float64
dtype: object

__Removing Dollar And Commas From The Budget:__

In [47]:
movie_dataframes['budget'] = movie_dataframes['budget'].str.replace(',', '')
movie_dataframes['budget'] = movie_dataframes['budget'].str.replace('$', '')

__Removing EUR From The Budget (If Any):__

In [49]:
movie_dataframes['budget'] = movie_dataframes['budget'].str.replace('EUR', '')

movie_dataframes

,movie_id,movie,year,imdb,metascore,votes,lengths,ratings,genres,budget,language,country,cumulative_worldwide_gross
0,8579674,1917,2019,8.6,79,64668,119,R,"Drama, War",100000000,"['English', 'French', 'German']","['UK', 'USA']","$143,336,984"
1,2527338,Star Wars: Episode IX - The Rise of Skywalker,2019,6.9,54,233251,142,PG-13,"Action, Adventure, Fantasy",200000000,['English'],['USA'],"$1,028,754,801"
2,7131622,Once Upon a Time... in Hollywood,2019,7.8,83,341435,161,R,"Comedy, Drama",90000000,"['English', 'Italian', 'Spanish']","['USA', 'UK', 'China']","$373,058,543"
3,7286456,Joker,2019,8.6,59,618468,122,R,"Crime, Drama, Thriller",55000000,['English'],"['Canada', 'USA']","$1,069,147,201"
4,8946378,Knives Out,2019,8.1,82,122999,131,PG-13,"Comedy, Crime, Drama",40000000,"['English', 'Spanish']",['USA'],"$277,992,387"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,947798,Black Swan,2010,8.0,79,667004,108,R,"Drama, Thriller",NaN,NaN,NaN,NaN
39996,1099212,Twilight,2008,5.2,56,403771,122,PG-13,"Drama, Fantasy, Romance",37000000,['English'],['USA'],"$393,853,583"
39997,5792656,Running with the Devil,2019,5.4,42,3242,100,R,"Crime, Drama, Thriller",NaN,"['English', 'Spanish']","['Colombia', 'USA']","$111,218"
39998,5177088,The Girl in the Spider's Web,2018,6.1,43,35174,115,R,"Action, Crime, Drama",43000000,['English'],"['Germany', 'Sweden', 'USA']","$35,164,920"


In [50]:
movie_dataframes['cumulative_worldwide_gross'] = movie_dataframes['cumulative_worldwide_gross '].str.replace(',', '')

In [51]:
movie_dataframes['cumulative_worldwide_gross'] = movie_dataframes['cumulative_worldwide_gross'].str.replace('$', '')
movie_dataframes['cumulative_worldwide_gross'] = movie_dataframes['cumulative_worldwide_gross'].str.replace(',', '')
movie_dataframes['cumulative_worldwide_gross'] = movie_dataframes['cumulative_worldwide_gross'].str.replace('EUR', '')

In [53]:
movie_dataframes=movie_dataframes.drop(columns='cumulative_worldwide_gross ')

In [54]:
movie_dataframes

,movie_id,movie,year,imdb,metascore,votes,lengths,ratings,genres,budget,language,country,cumulative_worldwide_gross
0,8579674,1917,2019,8.6,79,64668,119,R,"Drama, War",100000000,"['English', 'French', 'German']","['UK', 'USA']",143336984
1,2527338,Star Wars: Episode IX - The Rise of Skywalker,2019,6.9,54,233251,142,PG-13,"Action, Adventure, Fantasy",200000000,['English'],['USA'],1028754801
2,7131622,Once Upon a Time... in Hollywood,2019,7.8,83,341435,161,R,"Comedy, Drama",90000000,"['English', 'Italian', 'Spanish']","['USA', 'UK', 'China']",373058543
3,7286456,Joker,2019,8.6,59,618468,122,R,"Crime, Drama, Thriller",55000000,['English'],"['Canada', 'USA']",1069147201
4,8946378,Knives Out,2019,8.1,82,122999,131,PG-13,"Comedy, Crime, Drama",40000000,"['English', 'Spanish']",['USA'],277992387
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,947798,Black Swan,2010,8.0,79,667004,108,R,"Drama, Thriller",NaN,NaN,NaN,NaN
39996,1099212,Twilight,2008,5.2,56,403771,122,PG-13,"Drama, Fantasy, Romance",37000000,['English'],['USA'],393853583
39997,5792656,Running with the Devil,2019,5.4,42,3242,100,R,"Crime, Drama, Thriller",NaN,"['English', 'Spanish']","['Colombia', 'USA']",111218
39998,5177088,The Girl in the Spider's Web,2018,6.1,43,35174,115,R,"Action, Crime, Drama",43000000,['English'],"['Germany', 'Sweden', 'USA']",35164920


__Checking For Any Null Values:__

In [55]:
movie_dataframes.isnull().sum()

movie_id                          0
movie                             0
year                              0
imdb                              0
metascore                         0
votes                             0
lengths                           0
ratings                           0
genres                            0
budget                        18430
language                      13839
country                       13839
cumulative_worldwide_gross    15512
dtype: int64

In [56]:
movie_dataframes=movie_dataframes.dropna()

In [57]:
movie_dataframes.isnull().sum()

movie_id                      0
movie                         0
year                          0
imdb                          0
metascore                     0
votes                         0
lengths                       0
ratings                       0
genres                        0
budget                        0
language                      0
country                       0
cumulative_worldwide_gross    0
dtype: int64

In [25]:
movie_dataframes['budget'] = movie_dataframes['budget'].astype(int)

In [26]:
movie_dataframes['cumulative_worldwide_gross'] = movie_dataframes['cumulative_worldwide_gross'].astype(int)

In [65]:
movie_dataframes.dtypes

movie_id                        int64
movie                          object
year                            int64
imdb                          float64
metascore                       int64
votes                           int64
lengths                         int64
ratings                        object
genres                         object
budget                          int64
language                       object
country                        object
cumulative_worldwide_gross      int64
dtype: object

__Saving Cleaner Dataset To CSV File:__

In [24]:
movie_dataframes.to_csv('../movie_dataframe_completemovies_clean.csv')

__Reading Saved File:__

In [2]:
movie_dataframes=pd.read_csv('../movie_dataframe_completemovies_clean.csv')

In [3]:
movie_dataframes.dtypes

Unnamed: 0                      int64
movie_id                        int64
movie                          object
year                            int64
imdb                          float64
metascore                       int64
votes                           int64
lengths                         int64
ratings                        object
genres                         object
budget                          int64
language                       object
country                        object
cumulative_worldwide_gross      int64
dtype: object